In [4]:
import pandas as pd

def init_vote_result(vote_result_file, row_list):
    df_init = pd.DataFrame({
        "index": row_list,
        "vote_result": ["TestCase_4"] * len(row_list)
    })
    df_init.to_csv(vote_result_file, index=False)


def save_vote_result(vote_result_file, yes_list, result="TestCase_4"):
    df = pd.read_csv(vote_result_file)
    df.loc[df["index"].isin(yes_list), "vote_result"] = result
    df.to_csv(vote_result_file, index=False)


In [5]:
vote_result_file = 'output.csv'
row_list = [1,2,3,5,7,8]

init_vote_result(vote_result_file, row_list)


In [6]:
yes_list = [1,7,8]
save_vote_result(vote_result_file, yes_list, result="TestCase_2")

In [ ]:
# python
import shutil
import os

def copy_file(src, dst):
    # 确保目标目录存在
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    # shutil.copy2 会拷贝文件内容并尽量保留元数据（mtime, permissions 等）
    shutil.copy2(src, dst)

# 用法
proj_types = ['d4j_evo_prefix', 'gb_evo_prefix', 'd4j_llm_prefix', 'gb_llm_prefix']
methods = ['without_both', 'without_exception_inf', 'without_scenario_inf']
# generateds = ['togs_generated', 'togs_generated', 'togll_generated', 'toga_generated']

for method in methods:
    for proj_type in proj_types:
        src_file = f'../RQ3/{method}/{proj_type}/togs_generated/results/rq1_2.csv'
        dst_file = f'../RQ3/{method}/{proj_type}/rq1.csv'
        copy_file(src_file, dst_file)

In [ ]:
import pandas as pd
import os
def read_csv_second_row(file_path):
    df = pd.read_csv(file_path)
    if df.shape[0] < 2:
        return []  
    second_row = df.iloc[1, 1:]  
    return second_row.tolist()

def format_row(data):
    if not data or len(data) < 6:
        return None
    bugfound = int(data[0])
    fpr = round(float(data[1]) * 100, 2)
    precision = round(float(data[2]) * 100, 2)
    tp = int(data[3])
    fp = int(data[4])
    tn = int(data[5])
    return [bugfound, f"{fpr:.2f}%", f"{precision:.2f}%", tp, fp, tn]

def print_formatted_table(methods, proj_types):
    header = ['Method', 'BugFound', 'FPR', 'Precision', 'TP', 'FP', 'TN']
    method_w = max(len('Method'), max(len(m) for m in methods)) + 2
    col_w = 12
    col_w_l = 6
    fmt = f"{{:<{method_w}}}{{:>{col_w}}}{{:>{col_w}}}{{:>{col_w}}}{{:>{col_w_l}}}{{:>{col_w_l}}}{{:>{col_w_l}}}"

    for proj_type in proj_types:
        print(f"Project: {proj_type}")
        print(fmt.format(*header))
        for method in methods:
            file_path = f'../data/{proj_type}/{method}/rq1.csv'
            second_row_data = read_csv_second_row(file_path)
            formatted = format_row(second_row_data)
            if formatted is None:
                print(fmt.format(method, '-', '-', '-', '-', '-', '-'))
            else:
                # formatted = [BugFound, "FPR%", "Precision%", TP, FP, TN]
                print(fmt.format(method, formatted[0], formatted[1], formatted[2], formatted[3], formatted[4], formatted[5]))
        print()


print("RQ1_1 Results:")
rq1_1_proj_types = ['d4j_evo_prefix', 'gb_evo_prefix']
print_formatted_table(methods, rq1_1_proj_types)

print("RQ1_2 Results:")
rq1_2_proj_types = ['d4j_llm_prefix', 'gb_llm_prefix']
print_formatted_table(methods, rq1_2_proj_types)


RQ1_1 Results:
Project: d4j_evo_prefix
Method          BugFound         FPR   Precision    TP    FP    TN
deeporacle            53      21.56%      39.58%    76   116   422
llm_direct            42      25.29%      32.83%    65   133   393
togll                 18      43.77%      12.77%    36   246   316
toga                  46      23.98%      35.60%    68   123   390

Project: gb_evo_prefix
Method          BugFound         FPR   Precision    TP    FP    TN
deeporacle             8      26.67%      38.46%    15    24    66
llm_direct             4      32.18%      30.00%    12    28    59
togll                  1      37.21%      31.91%    15    32    54
toga                   5      23.23%      20.69%     6    23    76

RQ1_2 Results:
Project: d4j_llm_prefix
Method          BugFound         FPR   Precision    TP    FP    TN
deeporacle           138      18.00%      43.79%   261   335  1526
llm_direct           120      18.50%      38.82%   217   342  1507
togll                 34  

In [ ]:
import pandas as pd
import os


def read_second_row_vals(file_path, cols):
    if not os.path.exists(file_path):
        return {c: 0.0 for c in cols}
    df = pd.read_csv(file_path)
    if df.shape[0] < 2:
        return {c: 0.0 for c in cols}
    row = df.iloc[1]
    out = {}
    for c in cols:
        out[c] = float(row.get(c, 0.0))
    return out

def print_aggregated_results(methods, proj_types):
    cols = ['BugFound', 'TPs', 'FPs', 'TNs']

    method_w = max(len('Method'), max(len(m) for m in methods)) + 2
    col_w = 12
    col_w_l = 6
    fmt = f"{{:<{method_w}}}{{:>{col_w}}}{{:>{col_w}}}{{:>{col_w}}}{{:>{col_w_l}}}{{:>{col_w_l}}}{{:>{col_w_l}}}"

    print(fmt.format('Method', 'BugFound', 'FPR', 'Precision', 'TP', 'FP', 'TN'))
    for method in methods:
        totals = {c: 0.0 for c in cols}
        for proj in proj_types:
            fp = f'../RQ3/{method}/{proj}/rq1.csv'
            vals = read_second_row_vals(fp, cols)
            for c in cols:
                totals[c] += vals[c]
        bugfound = int(totals['BugFound'])
        tp = int(totals['TPs'])
        fp_cnt = int(totals['FPs'])
        tn = int(totals['TNs'])
        fpr = (fp_cnt / (fp_cnt + tn)) if (fp_cnt + tn) > 0 else 0.0
        precision = (tp / (tp + fp_cnt)) if (tp + fp_cnt) > 0 else 0.0
        print(fmt.format(method,
                        bugfound,
                        f"{fpr*100:.2f}%",
                        f"{precision*100:.2f}%",
                        tp, fp_cnt, tn))
    print()

if __name__ == "__main__":
    methods = ['deeporacle', 'without_exception_inf', 'without_scenario_inf', 'without_both']
    proj_types_evo = ['d4j_evo_prefix', 'gb_evo_prefix']
    print_aggregated_results(methods, proj_types_evo)

    proj_types_llm = ['d4j_llm_prefix', 'gb_llm_prefix']
    print_aggregated_results(methods, proj_types_llm)

Method                     BugFound         FPR   Precision    TP    FP    TN
deeporacle                       61      22.29%      39.39%    91   140   488
without_exception_inf            56      25.96%      34.80%    87   163   465
without_scenario_inf             43      25.20%      35.00%    84   156   463
without_both                     41      26.66%      32.93%    81   165   454

Method                     BugFound         FPR   Precision    TP    FP    TN
deeporacle                      153      18.39%      44.09%   291   369  1638
without_exception_inf           151      19.62%      42.31%   289   394  1614
without_scenario_inf            142      22.77%      35.76%   260   467  1584
without_both                    140      23.94%      34.53%   259   491  1560

